this notebook visualizes for each frame of the rescan: the predicted match between the segments and reference objects compared to the ground truth match that is excpected

In [ ]:
import pickle
import plyfile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import numpy as np
import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT' #adjust
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r, od3du_utils

#adjust the paths as needed
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes' 
new_scan_id =  "74ef846e-9dce-2d66-83d5-294aac7b1b0f" #choose whatever test rescan Id you want
curr_scan_id = scan3r.get_reference_id(data_dir, new_scan_id) #reference scene
patch_h= 18
image_height = 540
image_width = 960
patch_w = 32
patch_height = 30
patch_width = 30

In [ ]:
#this function creates a color image of the size 960x540 from the patches
def create_color_img_from_obj_id(data_dir,scan_id,obj_id_mat):
    #access the mesh file to get the colour of the ids
    colour_dict = od3du_utils.get_id_colours(data_dir, scan_id)
    #initialize the new matrix 
    og_height, og_width = obj_id_mat.shape
    new_height = og_height * patch_height
    new_width = og_width * patch_width

    colour_mat = np.zeros((new_height,new_width,3))

    #go over each element and assign the colour of the dictionary
    for h in range(og_height):
        for w in range(og_width):
            if obj_id_mat[h][w] in colour_dict.keys():
                colour = colour_dict[obj_id_mat[h][w]]
                colour_mat[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width] = colour


    return colour_mat

In [ ]:
#access the segmentation info along with the predicted matches
segmentation_info_path = osp.join(data_dir, "files/Segmentation/DinoV2/objects", new_scan_id + ".h5")
segmentation_info = od3du_utils.read_segmentation_data(segmentation_info_path)
id_matches = od3du_utils.read_matching_data(osp.join(data_dir, "files"), new_scan_id)

for frame, match_list in id_matches.items():
    # Extract and store the frame index and corresponding matches

    #generate a pixelwise image 
    new_img_pixelwise = od3du_utils.generate_pixel_level(segmentation_info[frame], match_list, image_height, image_width)

    #quantize to patches
    new_img_patchwise = od3du_utils.quantize_to_patch_level(new_img_pixelwise, image_height, image_width, patch_h, patch_w)
    #aggregate to patches and colour it, we want the colours which are used in curr_scan id / reference scene
    new_img_colour = create_color_img_from_obj_id(data_dir,curr_scan_id, new_img_patchwise)
 
    """
    access the gt of the image and get the accuracy of the prediction, print result and display 
    """
    #get the gt patches for the segmented scene! so for the dinov2 segmentation: but the colour will be based on the current scene
    gt_input_patchwise_path =  osp.join(data_dir,"files/patch_anno/patch_anno_32_18", new_scan_id + '.pkl')
    with open(gt_input_patchwise_path, 'rb') as file:
        gt_input_patchwise = pickle.load(file)
    gt_input_colour = create_color_img_from_obj_id(data_dir,curr_scan_id,gt_input_patchwise[frame])


    # """
    # display the newly computed images & GT next to each other
    # """

    #display the gt_image and the new patchwise image next to each other
    title1 = "gt_patces frame: " + frame
    title2 = "predicted_patches frame: " + frame

   # Create a blank canvas to combine images horizontally
    height = max(gt_input_colour.shape[0], new_img_colour.shape[0])  # Max height of both images
    width = gt_input_colour.shape[1] + new_img_colour.shape[1] + 20  # Total width of both images with a small gap
    combined_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Place images on the blank canvas
    combined_image[:gt_input_colour.shape[0], :gt_input_colour.shape[1]] = gt_input_colour
    combined_image[:new_img_colour.shape[0], gt_input_colour.shape[1] + 20:] = new_img_colour

    # Add titles to the images
    cv2.putText(combined_image, title1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(combined_image, title2, (gt_input_colour.shape[1] + 30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Use matplotlib to display the combined image
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')  # Hide axes
    plt.show()

